In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Who's the current President of France?")
print(results)

The current president is Emmanuel Macron, who succeeded François Hollande on 14 May 2017 following the 2017 presidential election, and was inaugurated for a second term on 7 May 2022 following the 2022 presidential election. May 31, 2025 · As of 2025, the President of France is Emmanuel Macron, a centrist leader known for his ambitious reforms, pro-European stance, and distinctive personality. This article explores who Emmanuel Macron is, his political path, and what kind of person and leader he has proven to be during his presidency. Jan 25, 2024 · - The Head of State is the President , and the current President is Emmanuel Macron. - The Head of the Government is the Prime Minister, and the current Prime Minister of France is Jean... Emmanuel Macron: Emmanuel Macron was elected President of France in 2017 and was re-elected in 2022. He is the current head of state of France . Angela Merkel: Angela Merkel served as the Chancellor of Germany from 2005 to 2021. She is not associated with

making tool example

In [3]:
from langchain_core.tools import Tool
import random

def get_weather(location: str)-> str:
    weather_conditions=[
        {"condition": "RAINY", "temp_c":15},
        {"condition": "CLEAR", "temp_c":25},
        {"condition": "WINDY", "temp_c":20},
    ]
    data= random.choice(weather_conditions)
    return f"Weather in {location} :{data['condition']}, {data['temp_c']} C"

weather_info_too= Tool(
    name="get_weateher",
    func= get_weather,
    description="Fetches dummy weather information for a given location."
)                                

In [5]:
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    try:
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            model_id = getattr(model, "modelId", getattr(model, "id", "Unknown"))
            downloads = getattr(model, "downloads", "Unknown")
            return f"The most downloaded model by {author} is {model_id} with {downloads:,} downloads"
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"

hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub."
)

print(hub_stats_tool.invoke("facebook"))


The most downloaded model by facebook is facebook/contriever with 7,619,207 downloads


Integrating the tools

In [6]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import sys

load_dotenv()

try:
    API_KEY = os.environ["GEMINI_PAID_KEY"]
except KeyError:
    print("Error: The 'gemini_api_key' environment variable is not set.")
    sys.exit(1)

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel(
    model_name="gemini-2.5-pro",        
)

In [ ]:
import os
from typing import TypedDict, List
from langgraph.graph import StateGraph, START
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
import google.generativeai as genai
from google.genai import types

load_dotenv()
try:
    GEMINI_API_KEY = os.environ["gemini_api_key"]
except KeyError:
    print("Error: The 'gemini_api_key' environment variable is not set.")
    exit(1)

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(model_name="gemini-2.5-pro")

class AgentState(TypedDict):
    messages: List[AnyMessage]

MAX_HISTORY = 3

def call_gemini(prompt: str, state: AgentState) -> AgentState:
    last_messages = state["messages"][-MAX_HISTORY:]
    contents = []

    for msg in last_messages:
        role = "assistant" if isinstance(msg, AIMessage) else "user"
        text = msg.content[:500]
        contents.append(
            types.Content(
                role=role,
                parts=[types.TextPart(text=text)]
            )
        )

    contents.append(
        types.Content(
            role="user",
            parts=[types.TextPart(text=prompt[:1000])]
        )
    )

    response = model.generate_content(
        contents=contents,
        response_parameters=types.ResponseParameters(max_output_tokens=500)
    )

    ai_text = response.text.strip()
    ai_msg = AIMessage(content=ai_text)
    return {"messages": state["messages"] + [ai_msg]}

def assistant(state: AgentState) -> AgentState:
    last_messages = "\n".join(msg.content for msg in state["messages"][-MAX_HISTORY:])
    prompt = f"As an assistant, respond to the following user query:\n\n{last_messages}"
    return call_gemini(prompt, state)


tools = {
    "search_tool": search_tool,
    "weather_tool": get_weather,
    "hub_stats_tool": hub_stats_tool
}

def route_tool(state: AgentState) -> str:
    last_msg = state["messages"][-1].content.lower()
    if "facebook" in last_msg or "popular model" in last_msg:
        return "hub_stats_tool"
    elif "weather" in last_msg:
        return "weather_tool"
    elif "who" in last_msg or "search" in last_msg:
        return "search_tool"
    else:
        return "assistant"

builder = StateGraph(AgentState)
builder.add_node("assistant", assistant)

for tool_name, tool_func in tools.items():
    builder.add_node(tool_name, tool_func)

builder.add_edge(START, "assistant")

builder.add_conditional_edges("assistant", route_tool, {
    "search_tool": "search_tool",
    "weather_tool": "weather_tool",
    "hub_stats_tool": "hub_stats_tool",
    "assistant": "assistant"
})

for tool_name in tools.keys():
    builder.add_edge(tool_name, "assistant")

alfred = builder.compile()

messages = [HumanMessage(content="Who is Facebook and what's their most popular model?")]
response = alfred.invoke({"messages": messages})

print("Alfred's Response:")
print(response["messages"][-1].content)

AttributeError: module 'google.genai.types' has no attribute 'TextPart'